In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

from tqdm import tqdm_notebook

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

### Model

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(5*5*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Preprocess

In [21]:
train_path = glob('../dataset/cifar/train/*.png')[:1000]
test_path = glob('../dataset/cifar/test/*.png')[:1000]
classes = np.unique([_.split('\\')[-1].split('_')[-1].replace('.png', '') for _ in train_path])
len(train_path), len(test_path)

(1000, 1000)

In [22]:
def get_label(path):
    lbl_name = os.path.basename(path).split('_')[-1].replace('.png','')
    label = np.argmax(classes == lbl_name)
    return label

class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):
        self.data_paths = data_paths
        self.transform = transform
    
    def __len__(self):
        return len(self.data_paths)
    
    def __getitem__(self, idx):
        path = self.data_paths[idx]
        
        # Read Image
        image = Image.open(path)
        
         # Get Label
        label = get_label(path)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [23]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

kwargs = {'num_workers' : 1, 'pin_memory' : True} if use_cuda else {}

In [24]:
train_loader = torch.utils.data.DataLoader(
    Dataset(train_path,
            transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean = [0.406],
                    std = [0.225]
                )
            ])),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_path,
            transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(
                    mean = [0.406],
                    std = [0.225]
                )
            ])),
    batch_size=batch_size,
    shuffle=False,
    **kwargs
)

### Optimizer

In [25]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(),
                      lr=lr,
                      momentum=momentum)

### Learning Rate Scheduler

In [33]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(optimizer,
                              mode='max',
                              factor=0.1,
                              patience=0,
                              verbose=True)

# step만큼 epoch이 진행될 경우 learning rate = learning rate * gamma
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer,
                                             step_size=2, gamma=0.1)

### Training

In [32]:
for epoch in range(1, 6):
    # Train mode
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # backpropagation 계산 전 0으로 기울기 초기화
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch : {} [{} / {} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()
            ))
            
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    # Learning Rate Scheduler
    # scheduler.step(accuracy)
    exp_lr_scheduler.step()
    
    print('\nTest set : Average loss : {:.4f}, Accuracy : {}/{} ({:.0f}%) lr : {}\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy, optimizer.param_groups[0]['lr']
    ))

Train Epoch : 1 [0 / 1000 (0%)]	Loss : 2.257652

Test set : Average loss : 2.2684, Accuracy : 159/1000 (16%) lr : 0.001

Train Epoch : 2 [0 / 1000 (0%)]	Loss : 2.251322

Test set : Average loss : 2.2647, Accuracy : 165/1000 (16%) lr : 0.0001

Train Epoch : 3 [0 / 1000 (0%)]	Loss : 2.265094

Test set : Average loss : 2.2643, Accuracy : 163/1000 (16%) lr : 0.0001

Train Epoch : 4 [0 / 1000 (0%)]	Loss : 2.256553

Test set : Average loss : 2.2639, Accuracy : 165/1000 (16%) lr : 1e-05

Train Epoch : 5 [0 / 1000 (0%)]	Loss : 2.257980

Test set : Average loss : 2.2639, Accuracy : 165/1000 (16%) lr : 1e-05

